In [1]:
# Imports 
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import cv2
import os
import json
import math
import torch.nn.init as init

# Set seed for randomize functions (Ez reproduction of results)
random.seed(100)

# Import TuSimple loader
import sys
sys.path.insert(0,'../resources/')
from tusimple import TuSimple
from mask_transformer import MaskTransformer

/home/dimitris/anaconda3/envs/py10/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/dimitris/anaconda3/envs/py10/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# ROOT DIRECTORIES
root_dir = os.path.dirname(os.getcwd())
annotated_dir = os.path.join(root_dir,'datasets/tusimple/train_set/annotations')
clips_dir = os.path.join(root_dir,'datasets/tusimple/train_set/')
annotated = os.listdir(annotated_dir)
    
# Get path directories for clips and annotations for the TUSimple dataset + ground truth dictionary
annotations = list()
for gt_file in annotated:
    path = os.path.join(annotated_dir,gt_file)
    json_gt = [json.loads(line) for line in open(path)]
    annotations.append(json_gt)
    
annotations = [a for f in annotations for a in f]

In [21]:
# Generate the mapping dict renaming the pretrained weights layers names to the desired format
def generate_mapping_dict():
    mapping = {
        'norm.bias' : 'norm.bias',
        'norm.weight': 'norm.weight',
        'pos_embed': 'pos_embedding',
        'patch_embed.proj.bias': 'patch_embedding.proj.bias', 
        'patch_embed.proj.weight': 'patch_embedding.proj.weight',
    }

    for i in range(12):
        prefix = f'blocks.{i}.'

        mapping[f'{prefix}norm1.bias'] = f'transformer.layers.{i}.norm1.bias'
        mapping[f'{prefix}norm1.weight'] = f'transformer.layers.{i}.norm1.weight'
        mapping[f'{prefix}norm2.bias'] = f'transformer.layers.{i}.norm2.bias'
        mapping[f'{prefix}norm2.weight'] = f'transformer.layers.{i}.norm2.weight'
        mapping[f'{prefix}mlp.fc1.bias'] = f'transformer.layers.{i}.linear1.bias'
        mapping[f'{prefix}mlp.fc1.weight'] = f'transformer.layers.{i}.linear1.weight'
        mapping[f'{prefix}mlp.fc2.bias'] = f'transformer.layers.{i}.linear2.bias' 
        mapping[f'{prefix}mlp.fc2.weight'] = f'transformer.layers.{i}.linear2.weight'
        mapping[f'{prefix}attn.proj.bias'] = f'transformer.layers.{i}.self_attn.out_proj.bias'
        mapping[f'{prefix}attn.proj.weight'] = f'transformer.layers.{i}.self_attn.out_proj.weight'
        mapping[f'{prefix}attn.qkv.bias'] = f'transformer.layers.{i}.self_attn.in_proj_bias'
        mapping[f'{prefix}attn.qkv.weight'] = f'transformer.layers.{i}.self_attn.in_proj_weight'

    return mapping

# Resize the pretrained positional embeddings to desired dimensions
def resizing_pos_pretrained(pretrained_dict, model_dict):
    pretrained_pos_embedding = pretrained_dict['pos_embedding']

    # Get new shape
    new_pos_embedding = torch.zeros_like(model_dict['pos_embedding'])
    new_shape = tuple(torch.zeros_like(model_dict['pos_embedding']).squeeze(0).shape)

    # Interpolate pre-trained embed to new size
    new_pos_embedding =  F.interpolate(pretrained_pos_embedding.unsqueeze(0), size= new_shape, mode='bilinear', align_corners=False).squeeze(0)

    # Update the state_dict with the resized pos_embedding tensor
    pretrained_dict['pos_embedding'] = new_pos_embedding
    return pretrained_dict

In [22]:
# Patch embedding class
class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, embed_dim, channels):
        super().__init__()

        self.image_size = image_size
        if image_size[0] % patch_size != 0 or image_size[1] % patch_size != 0:
            raise ValueError("image dimensions must be divisible by the patch size")
        self.grid_size = image_size[0] // patch_size, image_size[1] // patch_size
        self.num_patches = self.grid_size[0] * self.grid_size[1]
        self.patch_size = patch_size

        self.proj = nn.Conv2d(
            channels, embed_dim, kernel_size=patch_size, stride=patch_size
        )

    def forward(self, im):
        try:
            B, C, H, W = im.shape
        except:
            _, H, W = im.shape
        x = self.proj(im).flatten(2).transpose(1, 2)
        return x
    
    
    
# B-16 ViT Class
class ViT(nn.Module):
    def __init__(self, image_size=224, patch_size=16, num_classes=1000, dim=768, depth=12, heads=12, 
                 mlp_dim=3072, dropout=0.1,load_pre = False, pre_trained_path = None):
        super().__init__()
        self.image_size = image_size
        self.patch_size = patch_size
        self.num_classes = num_classes
        self.dim = dim
        self.depth = depth
        self.heads = heads
        self.mlp_dim = mlp_dim
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(dim)
        
        # Calculate the number of patches
        self.num_patches = (image_size // patch_size) ** 2
        patch_dim = 3 * patch_size ** 2

        # Define the patch embedding layer
        # self.patch_embedding = nn.Conv2d(in_channels=3, out_channels=dim, kernel_size=patch_size, stride=patch_size)
        self.patch_embedding = PatchEmbedding((self.image_size,self.image_size),self.patch_size,self.dim, 3)
        
        
        # Define the positional embedding layer
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches, dim))
        self.pos_embedding = nn.init.trunc_normal_(self.pos_embedding,std= 0.02)
        
        # Define the transformer layers
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout),
            num_layers=depth
        )
        
        # Load model with pre-trained weights if flas is true else initiallize randomly
        if load_pre:
            self.load_pretrained_weights(pre_trained_path)
        else:
            # Randomly initialize patch embedding layer weights
            init.normal_(self.patch_embedding.proj.weight, std=0.02)

            # Initialize transformer layer weights
            for i in range(depth):
                layer = self.transformer.layers[i]
                # Multi-Head Attention weights
                init.normal_(layer.self_attn.in_proj_weight, std=0.02)
                init.normal_(layer.self_attn.out_proj.weight, std=0.02)
                # Feed-Forward layer weights
                init.normal_(layer.linear1.weight, std=0.02)
                init.normal_(layer.linear2.weight, std=0.02)
                # Layer Normalization weights
                init.constant_(layer.norm1.weight, 1)
                init.constant_(layer.norm2.weight, 1)

    def forward(self, x, return_features = True):
        # Apply the patch embedding layer
        
        x = self.patch_embedding(x)
        
        # Reshape the patches            
        x = x.flatten(2).transpose(1, 2)
        
        # Resize positional embeddings
        if self.image_size != 224:
            resized_size = self.image_size
            self.resize_pos_embeds(resized_size)
            
        # Dynamically expand pos embed across batch dimension
        if self.training:
            pos_embedding = nn.Parameter(self.pos_embedding.expand(x.shape[0], -1, -1))
            # Add the positional embeddings and use dropout
            x = (x.reshape(x.shape[0], -1, 768) + pos_embedding)
            x = self.dropout(x)
        else:
            pos_embedding = self.pos_embedding
            # Add the positional embeddings and use dropout
            x = (x.reshape(1, -1, 768) + pos_embedding)
            x = self.dropout(x)
        
        # Apply the transformer layers
        x = self.transformer(x)
        
        # Apply layer normalization before returning the transformed features 
        x = self.norm(x)
        
        if return_features:
            return x

    # Resize pos embeddings functionality for tuning the ViT to accept resized images
    def resize_pos_embeds(self, new_image_size):
        # Get the original size of the positional embeddings
        orig_pos_embeds = self.pos_embedding

        # Calculate the number of patches for the new image size
        new_num_patches = (new_image_size // self.patch_size) ** 2

        # Define the new size of the positional embeddings based on the new number of patches
        new_embed_size = (new_num_patches, self.dim)  # Keep the same number of tokens
        new_pos_embeds = F.interpolate(orig_pos_embeds.unsqueeze(0), size=new_embed_size).squeeze(0)

        # Replace the original positional embeddings with the new ones
        self.pos_embedding = nn.Parameter(new_pos_embeds)
        
    # Load pre-trained weights method
    def load_pretrained_weights(self, pretrained_path):
        map_dict = generate_mapping_dict()
        pretrained = torch.load(pretrained_path)
        model_state_dict = self.state_dict()
    
        # create new state dict with mapped keys
        new_state_dict = {}
        for key in pretrained:
            if key in map_dict:
                new_state_dict[map_dict[key]] = pretrained[key]
            else:
                if key in model_state_dict:
                    new_state_dict[key] = pretrained[key]

         # Test and see if resizing these is a good idea else keep the original randomly initialized weights
        new_state_dict = resizing_pos_pretrained(new_state_dict,model_state_dict)  
        
        # Load the mapped weights into our ViT model
        self.load_state_dict(new_state_dict)
        print('Succesfully created ViT with pre-trained weights...!')

    

In [23]:
# Instantiate a ViT model with pre-trained weights
model = ViT(image_size=640, patch_size=16, num_classes=2, dim=768, depth=12, heads=12, 
            mlp_dim=3072, dropout=0.1, load_pre= True, pre_trained_path= '../pre-trained/jx_vit_base_p16_224-80ecf9dd.pth')

torch.Size([1, 197, 768])
tensor([[[ 0.0096,  0.0151, -0.2672,  ...,  0.0020, -0.0100, -0.4082],
         [ 0.1557, -0.1242,  0.4469,  ...,  0.0160, -0.0571, -0.0286],
         [ 0.0304, -0.0099,  0.2231,  ...,  0.0530,  0.0797,  0.0328],
         ...,
         [-0.0285, -0.0301, -0.3384,  ...,  0.0158,  0.0020, -0.0384],
         [-0.0883,  0.0620, -0.2314,  ...,  0.0584,  0.0558, -0.1647],
         [-0.0134,  0.0084,  0.0546,  ..., -0.0511, -0.1091, -0.1272]]])
(1600, 768)
tensor([[[ 0.0096,  0.0151, -0.2672,  ...,  0.0020, -0.0100, -0.4082],
         [ 0.0096,  0.0151, -0.2672,  ...,  0.0020, -0.0100, -0.4082],
         [ 0.0096,  0.0151, -0.2672,  ...,  0.0020, -0.0100, -0.4082],
         ...,
         [-0.0134,  0.0084,  0.0546,  ..., -0.0511, -0.1091, -0.1272],
         [-0.0134,  0.0084,  0.0546,  ..., -0.0511, -0.1091, -0.1272],
         [-0.0134,  0.0084,  0.0546,  ..., -0.0511, -0.1091, -0.1272]]])
Succesfully created ViT with pre-trained weights...!


In [6]:
dataset = TuSimple(train_annotations = annotations, train_img_dir = clips_dir, resize_to = (640,640), subset_size = 0.05)
    
img_tns, gt = dataset[0]

In [7]:
gt['gt_tensor'].unique()

tensor([0., 1.])

In [8]:
encoder_test = model(img_tns)
encoder_test.shape

torch.Size([1, 1600, 768])

In [9]:
masks_trans = MaskTransformer()
result = masks_trans(encoder_test)

In [10]:
result.shape

torch.Size([1, 2, 40, 40])

In [11]:
# Interpolate patch level class scores to pixel level class scores and use argmax to get match pixels with classes
mask = F.interpolate(result, size=(640, 640), mode="bilinear")
predicted_mask = torch.argmax(mask, dim=1).float()
predicted_mask.unique()

tensor([0., 1.])

In [12]:
# expand the output mask tensor along the channel dimension to match the ground truth tensor
predicted_mask = predicted_mask.unsqueeze(0).expand(3, -1, -1, -1).transpose(0, 1).squeeze(0)
predicted_mask = dataset.toImagearr(predicted_mask)
predicted_mask.shape

(640, 640, 3)

In [13]:
dataset.plot_img_gt(img_tns,predicted_mask)